In [28]:
import numpy as np
import os
from imutils import paths
import csv
import copy
from utils import get_hospital_dict, get_rale_dict, get_demograf_dict, shuffle, write_csv_file

In [39]:
metadata_file = "../data/Metadatos.csv"
#data_path = "../data/COVIDGR1.0-Segmentadas"
data_path = "../data/Revisadas-Clasificadas"

hospital_dict = get_hospital_dict(data_path, metadata_file)
niveles_rale_dict, puntos_rale_dict = get_rale_dict(data_path, metadata_file)
demograf_dict = get_demograf_dict(data_path, metadata_file)

In [16]:
def get_neg_pos(image_paths):
    neg = 0
    pos = 0
    for img in image_paths:
        label = img.split(os.sep)[-2]
        if label == 'N':
            neg = neg + 1
        else:
            pos = pos + 1
    
    return neg, pos


for k in ["San Cecilio", "MOTRIL", "ELCHE", "BAZA"]:
    image_paths = hospital_dict[k]
    neg, pos = get_neg_pos(image_paths)
    print(k + " N: " + str(neg) + " P: " + str(pos))

San Cecilio N: 426 P: 357
MOTRIL N: 13 P: 84
ELCHE N: 8 P: 24
BAZA N: 1 P: 20


In [11]:
federated_data = []
test_data = []
train_prop = 0.8
for k in ["San Cecilio", "MOTRIL", "ELCHE", "BAZA"]:
    image_paths = hospital_dict[k]
    image_paths = shuffle(image_paths)
    train_dim = int(train_prop * len(image_paths))
    train_image_paths = image_paths[0:train_dim]
    test_image_paths = image_paths[train_dim:]

    federated_data.append(train_image_paths)
    test_data.extend(test_image_paths)

csv_file_path = "../partitions/partition_noniid_hospital.csv"
write_csv_file(federated_data, test_data, csv_file_path)



In [48]:
neg_imgs = shuffle(copy.deepcopy(niveles_rale_dict["NEGATIVE"]))
num_pos = len(niveles_rale_dict["NORMAL-PCR+"]) + len(niveles_rale_dict["LEVE"]) + len(niveles_rale_dict["MODERADO"]) + len(niveles_rale_dict["GRAVE"])
num_neg = len(neg_imgs)
neg_imgs_dict = {}
for k in ["NORMAL-PCR+", "LEVE", "MODERADO", "GRAVE"]:
    num = int((len(niveles_rale_dict[k])/num_pos) * num_neg)
    a = []
    for i in range(num):
        a.append(neg_imgs[-1])
        neg_imgs.pop()
    neg_imgs_dict[k] = a

keys = ["NORMAL-PCR+", "LEVE", "MODERADO", "GRAVE"]
while neg_imgs:
    q = np.random.randint(1,5)
    neg_imgs_dict[keys[q]].append(neg_imgs[-1])
    neg_imgs.pop()

for k in neg_imgs_dict.keys():
    print(len(neg_imgs_dict[k]))


72
122
176
80


In [49]:
federated_data = []
test_data = []
train_prop = 0.8
for k in ["NORMAL-PCR+", "LEVE", "MODERADO", "GRAVE"]:
    pos_imgs = niveles_rale_dict[k]
    total = []
    total.extend(pos_imgs)
    total.extend(neg_imgs_dict[k])
    train_dim = int(train_prop * len(total))
    total = shuffle(total)
    train_image_paths = total[0:train_dim]
    test_image_paths = total[train_dim:]

    federated_data.append(train_image_paths)
    test_data.extend(test_image_paths)

csv_file_path = "../partitions/partition_noniid_nivelesrale_1.csv"
write_csv_file(federated_data, test_data, csv_file_path)

In [5]:
for k in niveles_rale_dict.keys():
    print(str(k) + ": " + str( len(niveles_rale_dict[k]) ) )

NEGATIVE: 450
NORMAL-PCR+: 79
LEVE: 133
MODERADO: 191
GRAVE: 87


In [6]:
for k in puntos_rale_dict.keys():
    print(str(k) + ": " + str( len(puntos_rale_dict[k]) ) )

0: 79
1: 62
2: 71
3: 71
4: 54
5: 64
6: 48
7: 25
8: 16
N: 450


In [9]:
for k in demograf_dict.keys():
    print(str(k) + ": " + str( len(demograf_dict[k]) ) )

Hombres<=50: 172
Mujeres<=50: 228
Hombres(50,65]: 135
Mujeres(50,65]: 148
Hombres>65: 158
Mujeres>65: 99
